## double

In [ ]:
"""ViT + CNN-CBAM + Fusion + Quantum Hybrid Network
ERBMAHE Dataset Classification
Binary Classification: Abnormal vs Normal

MODIFICATIONS:
- Using ORIGINAL 2D-CBAM (Channel + Spatial Attention on 2D feature maps)
- Train whole model from epoch 1 (no staged freezing)
- Keeps Focal Loss, EarlyStopping, CosineAnnealingLR, Quantum layer, everything else.
"""

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import PyTorch & torchvision
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from torchvision import transforms, models
    print("✓ PyTorch & torchvision imported successfully")
except Exception as e:
    print(f"❌ Error importing PyTorch/torchvision: {e}")
    sys.exit(1)

# Transformers (ViT)
try:
    from transformers import ViTForImageClassification, ViTImageProcessor, ViTModel
    print("✓ Transformers imported successfully")
except ImportError:
    print("📦 Installing Transformers...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
    from transformers import ViTForImageClassification, ViTImageProcessor, ViTModel
    print("✓ Transformers installed successfully")

# PennyLane (quantum)
try:
    import pennylane as qml
    print("✓ PennyLane imported successfully")
except ImportError:
    print("📦 Installing PennyLane...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pennylane"])
    import pennylane as qml
    print("✓ PennyLane installed successfully")

# Check GPU
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please run this script on a machine with CUDA-enabled GPU and proper drivers.")
device = torch.device('cuda')
print(f"🔧 Using device: {device}")
print(f"🔧 PyTorch version: {torch.__version__}")

# Config
MODEL_NAME = "google/vit-base-patch16-224"
CLASSES = ['Abnormal', 'Normal']
N_QUBITS = 4  # quantum qubits

# ============================================================================
# ORIGINAL 2D-CBAM (Channel Attention + Spatial Attention)
# ============================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(x_cat)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)
    
    def forward(self, x):
        # Channel attention
        x = x * self.channel_attention(x)
        # Spatial attention
        x = x * self.spatial_attention(x)
        return x

# ============================================================================
# CNN + CBAM BRANCH (ResNet18 backbone -> CBAM -> Global Pool)
# ============================================================================
class CNN_CBAM_Branch(nn.Module):
    def __init__(self, pretrained=True):
        super(CNN_CBAM_Branch, self).__init__()
        resnet = models.resnet18(pretrained=pretrained)
        # Extract features up to layer4 (before avgpool)
        # ResNet structure: conv1, bn1, relu, maxpool, layer1, layer2, layer3, layer4, avgpool, fc
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])  # up to layer4 (outputs 512 x 7 x 7)
        
        # Apply CBAM on the 2D feature maps
        self.cbam = CBAM(512, reduction=16, kernel_size=7)
        
        # Global average pooling to get 1D features
        self.global_pool = nn.AdaptiveAvgPool2d(1)
    
    def forward(self, x):
        # x: (B, 3, H, W)
        feat = self.backbone(x)          # (B, 512, 7, 7)
        feat = self.cbam(feat)           # (B, 512, 7, 7) with attention applied
        feat = self.global_pool(feat)    # (B, 512, 1, 1)
        feat = feat.view(feat.size(0), -1)  # (B, 512)
        return feat

# ============================================================================
# Quantum Circuit & Layer
# ============================================================================
class QuantumCircuit:
    def __init__(self, n_qubits=4):
        self.n_qubits = n_qubits
        self.dev = qml.device('default.qubit', wires=n_qubits)
    
    def circuit(self, inputs, weights):
        for i in range(self.n_qubits):
            qml.RX(inputs[i], wires=i)
            qml.RZ(inputs[i], wires=i)
        for i in range(self.n_qubits - 1):
            qml.CRX(weights[i], wires=[i, i+1])
        qml.CRX(weights[self.n_qubits-1], wires=[self.n_qubits-1, 0])
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def create_qnode(self, weights):
        @qml.qnode(self.dev, interface='torch')
        def qnode(inputs):
            return self.circuit(inputs, weights)
        return qnode

class QuantumLayer(nn.Module):
    def __init__(self, input_dim, n_qubits=4):
        super(QuantumLayer, self).__init__()
        self.n_qubits = n_qubits
        self.feature_compress = nn.Linear(input_dim, n_qubits)
        self.q_weights = nn.Parameter(torch.randn(n_qubits) * 0.01)
        self.qc = QuantumCircuit(n_qubits)
        self.feature_expand = nn.Linear(n_qubits, input_dim)
        self.skip_alpha = nn.Parameter(torch.tensor(0.1))
    
    def forward(self, x):
        identity = x
        batch_size = x.size(0)
        x_compressed = self.feature_compress(x)  # (B, n_qubits)
        qnode = self.qc.create_qnode(self.q_weights)
        q_out_list = []
        for i in range(batch_size):
            q_input = torch.tanh(x_compressed[i]) * np.pi
            q_result = torch.stack(qnode(q_input))
            q_result = q_result.float()
            q_out_list.append(q_result)
        q_out = torch.stack(q_out_list)  # (B, n_qubits)
        output = self.feature_expand(q_out)  # (B, input_dim)
        output = output + self.skip_alpha * identity
        return output

# ============================================================================
# Fusion Hybrid Model (ViT + CNN-CBAM -> Fusion -> Quantum -> Classifier)
# ============================================================================
class ViTQuantumHybrid(nn.Module):
    def __init__(self, model_name, num_classes=2, n_qubits=4):
        super(ViTQuantumHybrid, self).__init__()
        # ViT backbone
        self.vit = ViTModel.from_pretrained(model_name)
        vit_dim = self.vit.config.hidden_size  # typically 768
        
        # CNN branch with 2D-CBAM
        self.cnn_branch = CNN_CBAM_Branch(pretrained=True)
        cnn_dim = 512
        
        # Fusion projection
        self.fusion_proj = nn.Linear(vit_dim + cnn_dim, vit_dim)
        
        # Quantum enhancement
        self.quantum_layer = QuantumLayer(vit_dim, n_qubits=n_qubits)
        
        # Classifier
        self.classifier = nn.Sequential(
            nn.Linear(vit_dim, vit_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(vit_dim // 2, num_classes)
        )
    
    def forward(self, pixel_values):
        """
        pixel_values: tensor (B, 3, H, W) — produced by ViTImageProcessor
        """
        # ViT features
        vit_out = self.vit(pixel_values=pixel_values)
        vit_features = vit_out.last_hidden_state[:, 0]  # (B, vit_dim)

        # CNN features with 2D-CBAM attention
        cnn_features = self.cnn_branch(pixel_values)    # (B, 512)

        # Fuse and project
        fused = torch.cat([vit_features, cnn_features], dim=1)  # (B, vit_dim + 512)
        fused = self.fusion_proj(fused)  # (B, vit_dim)

        # Quantum enhancement
        quantum_features = self.quantum_layer(fused)  # (B, vit_dim)
        
        # Classification
        logits = self.classifier(quantum_features)
        return logits

# ============================================================================
# Focal Loss
# ============================================================================
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        p_t = torch.exp(-ce_loss)
        focal_loss = (1 - p_t) ** self.gamma * ce_loss
        
        if self.alpha is not None:
            if isinstance(self.alpha, (float, int)):
                alpha_t = self.alpha
            else:
                alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss
        
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# ============================================================================
# Early Stopping
# ============================================================================
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0, mode='max', verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_epoch = 0
    
    def __call__(self, current_score, epoch):
        if self.best_score is None:
            self.best_score = current_score
            self.best_epoch = epoch
            if self.verbose:
                print(f"  ✓ Initial best score: {current_score:.4f}")
            return False
        
        if self.mode == 'max':
            improved = current_score > (self.best_score + self.min_delta)
        else:
            improved = current_score < (self.best_score - self.min_delta)
        
        if improved:
            self.best_score = current_score
            self.best_epoch = epoch
            self.counter = 0
            if self.verbose:
                print(f"  ✓ New best score: {current_score:.4f}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"  No improvement. Patience: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f"\n⚠️ Early stopping triggered!")
                    print(f"   Best score: {self.best_score:.4f} at epoch {self.best_epoch}")
        return self.early_stop

# ============================================================================
# DATASET
# ============================================================================
class ERBMAHEDataset(Dataset):
    def __init__(self, dataframe, processor, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.processor = processor
        self.augment = augment
        self.aug_transform = transforms.Compose([
            transforms.RandomRotation(10),
            transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
            transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.0),
            transforms.RandomHorizontalFlip(p=0.5),
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'image_path']
        label = self.df.loc[idx, 'class_id']
        image = Image.open(img_path).convert('RGB')
        
        if self.augment:
            image = self.aug_transform(image)
        
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        return pixel_values, label

# ============================================================================
# TRAIN / VALIDATION FUNCTIONS
# ============================================================================

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    pbar = tqdm(dataloader, desc='Training')
    for pixel_values, labels in pbar:
        pixel_values, labels = pixel_values.to(device), labels.to(device)
        
        optimizer.zero_grad()
        logits = model(pixel_values)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        
        pbar.set_postfix({
            'loss': f'{running_loss/(pbar.n+1):.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    
    _, _, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)
    return running_loss / len(dataloader), 100. * correct / total, f1

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for pixel_values, labels in pbar:
            pixel_values, labels = pixel_values.to(device), labels.to(device)
            logits = model(pixel_values)
            loss = criterion(logits, labels)
            
            running_loss += loss.item()
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            
            pbar.set_postfix({
                'loss': f'{running_loss/(pbar.n+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    
    _, _, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)
    return running_loss / len(dataloader), 100. * correct / total, f1, all_preds, all_labels

# ============================================================================
# MAIN
# ============================================================================

def main():
    print("="*70)
    print("ViT + CNN-2D-CBAM + Fusion + Quantum Hybrid Network")
    print("ERBMAHE Dataset Classification")
    print("Binary Classification: Abnormal vs Normal")
    print("USING ORIGINAL 2D-CBAM + FOCAL LOSS + QUANTUM ENHANCEMENT")
    print("="*70)

    # Dataset path (update if needed)
    data_path = 'D:/training/archive/ICMR_datasets_ERBMAHE'

    print(f"\n🤖 Base Model: {MODEL_NAME}")
    print(f"⚛️  Quantum Qubits: {N_QUBITS}")
    print(f"📊 Classes: {CLASSES}")

    print("\n📁 Loading dataset...")
    data_list = []
    for class_name in CLASSES:
        class_path = os.path.join(data_path, class_name)
        if os.path.exists(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    data_list.append({
                        'image_path': os.path.join(class_path, img_file),
                        'label': class_name,
                        'class_id': CLASSES.index(class_name)
                    })
        else:
            print(f"⚠️ Warning: {class_path} not found!")
    
    df = pd.DataFrame(data_list)
    print(f"📊 Total images: {len(df)}")
    print("\nClass distribution:")
    print(df['label'].value_counts())

    # Class weights for focal loss
    class_counts = df['label'].value_counts().sort_index().values
    total_samples = len(df)
    class_weights = total_samples / (len(CLASSES) * class_counts)
    class_weights = torch.FloatTensor(class_weights).to(device)
    print(f"\n⚖️ Class weights for Focal Loss:")
    for i, class_name in enumerate(CLASSES):
        print(f"  {class_name}: {class_weights[i]:.4f}")

    # Splits
    temp_df, test_df = train_test_split(df, test_size=0.10, stratify=df['class_id'], random_state=42)
    train_df, val_df = train_test_split(temp_df, test_size=0.111111, stratify=temp_df['class_id'], random_state=42)
    print(f"\n📊 Dataset split:")
    print(f"  Train: {len(train_df)} ({len(train_df)/len(df)*100:.1f}%)")
    print(f"  Val:   {len(val_df)} ({len(val_df)/len(df)*100:.1f}%)")
    print(f"  Test:  {len(test_df)} ({len(test_df)/len(df)*100:.1f}%)")

    # ViT processor
    print(f"\n🔧 Loading ViT processor...")
    processor = ViTImageProcessor.from_pretrained(MODEL_NAME)

    # Datasets & dataloaders
    train_dataset = ERBMAHEDataset(train_df, processor=processor, augment=True)
    val_dataset = ERBMAHEDataset(val_df, processor=processor, augment=False)
    test_dataset = ERBMAHEDataset(test_df, processor=processor, augment=False)
    
    batch_size = 16
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)

    # Create model
    print(f"\n🤖 Creating Hybrid ViT + CNN-2D-CBAM + Fusion + Quantum Model...")
    model = ViTQuantumHybrid(model_name=MODEL_NAME, num_classes=len(CLASSES), n_qubits=N_QUBITS).to(device)

    # Ensure whole model is trainable from epoch 1
    for p in model.parameters():
        p.requires_grad = True

    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    quantum_params = sum(p.numel() for p in model.quantum_layer.parameters())

    print(f"✓ Total parameters: {total_params:,}")
    print(f"✓ Trainable parameters: {trainable_params:,}")
    print(f"✓ Quantum layer parameters: {quantum_params:,}")
    print(f"✓ Model size: {total_params * 4 / 1024 / 1024:.2f} MB")

    # Criterion
    criterion = FocalLoss(alpha=class_weights, gamma=2.0, reduction='mean')

    # Build optimizer
    def build_optimizer(model):
        groups = []
        # ViT params
        vit_params = [p for p in model.vit.parameters() if p.requires_grad]
        if vit_params:
            groups.append({'params': vit_params, 'lr': 1e-5})
        # CNN branch
        cnn_params = [p for p in model.cnn_branch.parameters() if p.requires_grad]
        if cnn_params:
            groups.append({'params': cnn_params, 'lr': 2e-4})
        # Quantum layer
        q_params = [p for p in model.quantum_layer.parameters() if p.requires_grad]
        if q_params:
            groups.append({'params': q_params, 'lr': 5e-5})
        # Classifier
        clf_params = [p for p in model.classifier.parameters() if p.requires_grad]
        if clf_params:
            groups.append({'params': clf_params, 'lr': 1e-4})
        # Fallback
        if not groups:
            groups = [{'params': [p for p in model.parameters() if p.requires_grad], 'lr': 1e-4}]
        opt = torch.optim.AdamW(groups, weight_decay=0.01)
        return opt

    optimizer = build_optimizer(model)
    scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50, eta_min=1e-6)
    early_stopping = EarlyStopping(patience=10, min_delta=0.001, mode='max', verbose=True)

    num_epochs = 50
    best_val_acc = 0.0

    print(f"\n⚙️ Training configuration:")
    print(f"  Epochs: {num_epochs}")
    print(f"  Batch size: {batch_size}")
    print(f"  Training: Whole model from epoch 1")
    print(f"  CBAM Type: Original 2D-CBAM (Channel + Spatial Attention)")
    print(f"  Learning rate (ViT backbone): 1e-5")
    print(f"  Learning rate (CNN backbone): 2e-4")
    print(f"  Learning rate (Quantum layer): 5e-5")
    print(f"  Learning rate (Classifier): 1e-4")
    print(f"  Scheduler: CosineAnnealingLR")
    print(f"  Early stopping patience: {early_stopping.patience}")
    print(f"  Loss function: Focal Loss (gamma=2.0)")

    # History
    history = {
        'train_loss': [], 'train_acc': [], 'train_f1': [],
        'val_loss': [], 'val_acc': [], 'val_f1': []
    }

    # TRAINING LOOP
    for epoch in range(num_epochs):
        print(f"\nEpoch {epoch+1}/{num_epochs}")
        print("-" * 70)

        train_loss, train_acc, train_f1 = train_epoch(model, train_loader, criterion, optimizer, device)
        val_loss, val_acc, val_f1, _, _ = validate(model, val_loader, criterion, device)

        scheduler.step()

        history['train_loss'].append(train_loss)
        history['train_acc'].append(train_acc)
        history['train_f1'].append(train_f1)
        history['val_loss'].append(val_loss)
        history['val_acc'].append(val_acc)
        history['val_f1'].append(val_f1)

        current_lr = optimizer.param_groups[0]['lr']
        print(f"\nResults:")
        print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%, F1: {train_f1:.4f}")
        print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%, F1: {val_f1:.4f}")
        print(f"  Learning Rate (group0): {current_lr:.2e}")

        if val_acc > best_val_acc:
            best_val_acc = val_acc
            torch.save(model.state_dict(), 'vit_quantum_hybrid_2d_cbam_best.pth')
            print(f"  💾 Best model saved! (Val Acc: {best_val_acc:.2f}%)")

        if early_stopping(val_acc, epoch+1):
            print(f"\n🛑 Training stopped early at epoch {epoch+1}")
            print(f"   Best validation accuracy: {early_stopping.best_score:.2f}% at epoch {early_stopping.best_epoch}")
            break

    # TEST EVALUATION
    print("\n" + "="*70)
    print("📊 EVALUATING ON TEST SET")
    print("="*70)
    model.load_state_dict(torch.load('vit_quantum_hybrid_2d_cbam_best.pth'))
    test_loss, test_acc, test_f1, y_pred, y_true = validate(model, test_loader, criterion, device)

    precision, recall, f1, support = precision_recall_fscore_support(y_true, y_pred, average='weighted')

    print(f"\n📈 Test Results:")
    print(f"  Accuracy:  {test_acc:.2f}%")
    print(f"  Precision: {precision:.4f}")
    print(f"  Recall:    {recall:.4f}")
    print(f"  F1 Score:  {f1:.4f}")

    # Per-class metrics
    print("\n📊 Per-Class Performance:")
    p_class, r_class, f1_class, s_class = precision_recall_fscore_support(y_true, y_pred, average=None)
    for i, class_name in enumerate(CLASSES):
        print(f"\n{class_name}:")
        print(f"  Precision: {p_class[i]:.4f}")
        print(f"  Recall:    {r_class[i]:.4f}")
        print(f"  F1 Score:  {f1_class[i]:.4f}")
        print(f"  Support:   {s_class[i]}")

    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
    plt.title('ViT + CNN-2D-CBAM + Fusion + Quantum - Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.tight_layout()
    plt.savefig('vit_quantum_2d_cbam_confusion_matrix.png', dpi=150)
    print("\n✓ Confusion matrix saved")

    # Training history plots
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    
    axes[0].plot(history['train_loss'], label='Train', marker='o')
    axes[0].plot(history['val_loss'], label='Val', marker='s')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training and Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)

    axes[1].plot(history['train_acc'], label='Train', marker='o')
    axes[1].plot(history['val_acc'], label='Val', marker='s')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Accuracy (%)')
    axes[1].set_title('Training and Validation Accuracy')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)

    axes[2].plot(history['train_f1'], label='Train', marker='o')
    axes[2].plot(history['val_f1'], label='Val', marker='s')
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('F1 Score')
    axes[2].set_title('Training and Validation F1 Score')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig('vit_quantum_2d_cbam_training_history.png', dpi=150)
    print("✓ Training history saved")

    # Specificity calculation
    tn = cm[0, 0]
    fp = cm[0, 1]
    fn = cm[1, 0]
    tp = cm[1, 1]
    specificity_normal = tn / (tn + fp) if (tn + fp) > 0 else 0
    specificity_abnormal = tp / (tp + fn) if (tp + fn) > 0 else 0
    avg_specificity = (specificity_normal + specificity_abnormal) / 2

    print(f"\n📊 Additional Metrics:")
    print(f"  Specificity (Normal):   {specificity_normal:.4f}")
    print(f"  Specificity (Abnormal): {specificity_abnormal:.4f}")
    print(f"  Average Specificity:    {avg_specificity:.4f}")

    print("\n" + "="*70)
    print("✅ TRAINING COMPLETE")
    print("="*70)
    print(f"\n📊 Final Summary:")
    print(f"  Model: ViT + CNN-2D-CBAM + Fusion + Quantum ({N_QUBITS} qubits)")
    print(f"  CBAM Type: Original 2D-CBAM (Channel + Spatial Attention)")
    print(f"  Dataset: ERBMAHE (Abnormal vs Normal)")
    print(f"  Loss Function: Focal Loss (gamma=2.0)")
    print(f"  Test Accuracy: {test_acc:.2f}%")
    print(f"  Test F1 Score: {test_f1:.4f}")
    print(f"  Best Val Accuracy: {best_val_acc:.2f}%")
    print(f"  Total Parameters: {total_params:,}")
    print(f"  Quantum Parameters: {quantum_params:,}")
    print("\n" + "="*70)

if __name__ == '__main__':
    main()

## KFOLD 2DCBAM

In [1]:
"""ViT + CNN-CBAM + Fusion + Quantum Hybrid Network
ERBMAHE Dataset Classification
Binary Classification: Abnormal vs Normal

METHOD 2 K-FOLD WITH SEPARATE TEST SET:
- 10% holdout test set (never used during training)
- K-Fold cross-validation on remaining 90%
- Validation-based early stopping
- Best fold model evaluated on test set
"""

import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'
os.environ['PYTORCH_CUDA_ALLOC_CONF'] = 'expandable_segments:True'

import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image
from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn.metrics import accuracy_score, precision_recall_fscore_support, confusion_matrix
import seaborn as sns
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

# Import PyTorch & torchvision
try:
    import torch
    import torch.nn as nn
    import torch.nn.functional as F
    from torch.utils.data import Dataset, DataLoader
    from torchvision import transforms, models
    print("✓ PyTorch & torchvision imported successfully")
except Exception as e:
    print(f"❌ Error importing PyTorch/torchvision: {e}")
    sys.exit(1)

# Transformers (ViT)
try:
    from transformers import ViTForImageClassification, ViTImageProcessor, ViTModel
    print("✓ Transformers imported successfully")
except ImportError:
    print("📦 Installing Transformers...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "transformers"])
    from transformers import ViTForImageClassification, ViTImageProcessor, ViTModel
    print("✓ Transformers installed successfully")

# PennyLane (quantum)
try:
    import pennylane as qml
    print("✓ PennyLane imported successfully")
except ImportError:
    print("📦 Installing PennyLane...")
    import subprocess
    subprocess.check_call([sys.executable, "-m", "pip", "install", "pennylane"])
    import pennylane as qml
    print("✓ PennyLane installed successfully")

# Check GPU
if not torch.cuda.is_available():
    raise RuntimeError("CUDA is not available. Please run this script on a machine with CUDA-enabled GPU and proper drivers.")
device = torch.device('cuda')
print(f"🔧 Using device: {device}")
print(f"🔧 PyTorch version: {torch.__version__}")

# Config
MODEL_NAME = "google/vit-base-patch16-224"
CLASSES = ['Abnormal', 'Normal']
N_QUBITS = 4
N_FOLDS = 5

# ============================================================================
# ORIGINAL 2D-CBAM (Channel Attention + Spatial Attention)
# ============================================================================
class ChannelAttention(nn.Module):
    def __init__(self, in_channels, reduction=16):
        super(ChannelAttention, self).__init__()
        self.avg_pool = nn.AdaptiveAvgPool2d(1)
        self.max_pool = nn.AdaptiveMaxPool2d(1)
        
        self.fc = nn.Sequential(
            nn.Conv2d(in_channels, in_channels // reduction, 1, bias=False),
            nn.ReLU(inplace=True),
            nn.Conv2d(in_channels // reduction, in_channels, 1, bias=False)
        )
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = self.fc(self.avg_pool(x))
        max_out = self.fc(self.max_pool(x))
        out = avg_out + max_out
        return self.sigmoid(out)

class SpatialAttention(nn.Module):
    def __init__(self, kernel_size=7):
        super(SpatialAttention, self).__init__()
        padding = (kernel_size - 1) // 2
        self.conv = nn.Conv2d(2, 1, kernel_size, padding=padding, bias=False)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        avg_out = torch.mean(x, dim=1, keepdim=True)
        max_out, _ = torch.max(x, dim=1, keepdim=True)
        x_cat = torch.cat([avg_out, max_out], dim=1)
        out = self.conv(x_cat)
        return self.sigmoid(out)

class CBAM(nn.Module):
    def __init__(self, in_channels, reduction=16, kernel_size=7):
        super(CBAM, self).__init__()
        self.channel_attention = ChannelAttention(in_channels, reduction)
        self.spatial_attention = SpatialAttention(kernel_size)
    
    def forward(self, x):
        x = x * self.channel_attention(x)
        x = x * self.spatial_attention(x)
        return x

# ============================================================================
# CNN + CBAM BRANCH
# ============================================================================
class CNN_CBAM_Branch(nn.Module):
    def __init__(self, pretrained=True):
        super(CNN_CBAM_Branch, self).__init__()
        resnet = models.resnet18(pretrained=pretrained)
        self.backbone = nn.Sequential(*list(resnet.children())[:-2])
        self.cbam = CBAM(512, reduction=16, kernel_size=7)
        self.global_pool = nn.AdaptiveAvgPool2d(1)
    
    def forward(self, x):
        feat = self.backbone(x)
        feat = self.cbam(feat)
        feat = self.global_pool(feat)
        feat = feat.view(feat.size(0), -1)
        return feat

# ============================================================================
# Quantum Circuit & Layer
# ============================================================================
class QuantumCircuit:
    def __init__(self, n_qubits=4):
        self.n_qubits = n_qubits
        self.dev = qml.device('default.qubit', wires=n_qubits)
    
    def circuit(self, inputs, weights):
        for i in range(self.n_qubits):
            qml.RX(inputs[i], wires=i)
            qml.RZ(inputs[i], wires=i)
        for i in range(self.n_qubits - 1):
            qml.CRX(weights[i], wires=[i, i+1])
        qml.CRX(weights[self.n_qubits-1], wires=[self.n_qubits-1, 0])
        return [qml.expval(qml.PauliZ(i)) for i in range(self.n_qubits)]
    
    def create_qnode(self, weights):
        @qml.qnode(self.dev, interface='torch')
        def qnode(inputs):
            return self.circuit(inputs, weights)
        return qnode

class QuantumLayer(nn.Module):
    def __init__(self, input_dim, n_qubits=4):
        super(QuantumLayer, self).__init__()
        self.n_qubits = n_qubits
        self.feature_compress = nn.Linear(input_dim, n_qubits)
        self.q_weights = nn.Parameter(torch.randn(n_qubits) * 0.01)
        self.qc = QuantumCircuit(n_qubits)
        self.feature_expand = nn.Linear(n_qubits, input_dim)
        self.skip_alpha = nn.Parameter(torch.tensor(0.1))
    
    def forward(self, x):
        identity = x
        batch_size = x.size(0)
        x_compressed = self.feature_compress(x)
        qnode = self.qc.create_qnode(self.q_weights)
        q_out_list = []
        for i in range(batch_size):
            q_input = torch.tanh(x_compressed[i]) * np.pi
            q_result = torch.stack(qnode(q_input))
            q_result = q_result.float()
            q_out_list.append(q_result)
        q_out = torch.stack(q_out_list)
        output = self.feature_expand(q_out)
        output = output + self.skip_alpha * identity
        return output

# ============================================================================
# Fusion Hybrid Model
# ============================================================================
class ViTQuantumHybrid(nn.Module):
    def __init__(self, model_name, num_classes=2, n_qubits=4):
        super(ViTQuantumHybrid, self).__init__()
        self.vit = ViTModel.from_pretrained(model_name)
        vit_dim = self.vit.config.hidden_size
        self.cnn_branch = CNN_CBAM_Branch(pretrained=True)
        cnn_dim = 512
        self.fusion_proj = nn.Linear(vit_dim + cnn_dim, vit_dim)
        self.quantum_layer = QuantumLayer(vit_dim, n_qubits=n_qubits)
        self.classifier = nn.Sequential(
            nn.Linear(vit_dim, vit_dim // 2),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(vit_dim // 2, num_classes)
        )
    
    def forward(self, pixel_values):
        vit_out = self.vit(pixel_values=pixel_values)
        vit_features = vit_out.last_hidden_state[:, 0]
        cnn_features = self.cnn_branch(pixel_values)
        fused = torch.cat([vit_features, cnn_features], dim=1)
        fused = self.fusion_proj(fused)
        quantum_features = self.quantum_layer(fused)
        logits = self.classifier(quantum_features)
        return logits

# ============================================================================
# Focal Loss
# ============================================================================
class FocalLoss(nn.Module):
    def __init__(self, alpha=None, gamma=2.0, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    
    def forward(self, inputs, targets):
        ce_loss = F.cross_entropy(inputs, targets, reduction='none')
        p_t = torch.exp(-ce_loss)
        focal_loss = (1 - p_t) ** self.gamma * ce_loss
        if self.alpha is not None:
            if isinstance(self.alpha, (float, int)):
                alpha_t = self.alpha
            else:
                alpha_t = self.alpha[targets]
            focal_loss = alpha_t * focal_loss
        if self.reduction == 'mean':
            return focal_loss.mean()
        elif self.reduction == 'sum':
            return focal_loss.sum()
        else:
            return focal_loss

# ============================================================================
# Early Stopping
# ============================================================================
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0, mode='max', verbose=True):
        self.patience = patience
        self.min_delta = min_delta
        self.mode = mode
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.best_epoch = 0
    
    def __call__(self, current_score, epoch):
        if self.best_score is None:
            self.best_score = current_score
            self.best_epoch = epoch
            if self.verbose:
                print(f"  ✓ Initial best score: {current_score:.4f}")
            return False
        if self.mode == 'max':
            improved = current_score > (self.best_score + self.min_delta)
        else:
            improved = current_score < (self.best_score - self.min_delta)
        if improved:
            self.best_score = current_score
            self.best_epoch = epoch
            self.counter = 0
            if self.verbose:
                print(f"  ✓ New best score: {current_score:.4f}")
        else:
            self.counter += 1
            if self.verbose:
                print(f"  No improvement. Patience: {self.counter}/{self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
                if self.verbose:
                    print(f"\n⚠️ Early stopping triggered!")
                    print(f"   Best score: {self.best_score:.4f} at epoch {self.best_epoch}")
        return self.early_stop

# ============================================================================
# DATASET
# ============================================================================
class ERBMAHEDataset(Dataset):
    def __init__(self, dataframe, processor, augment=False):
        self.df = dataframe.reset_index(drop=True)
        self.processor = processor
        self.augment = augment
        self.aug_transform = transforms.Compose([
            transforms.RandomRotation(10),
            transforms.RandomAffine(degrees=0, translate=(0.05, 0.05)),
            transforms.ColorJitter(brightness=0.05, contrast=0.05, saturation=0.0),
            transforms.RandomHorizontalFlip(p=0.5),
        ])
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img_path = self.df.loc[idx, 'image_path']
        label = self.df.loc[idx, 'class_id']
        image = Image.open(img_path).convert('RGB')
        if self.augment:
            image = self.aug_transform(image)
        inputs = self.processor(images=image, return_tensors="pt")
        pixel_values = inputs['pixel_values'].squeeze(0)
        return pixel_values, label

# ============================================================================
# TRAIN / VALIDATION FUNCTIONS
# ============================================================================

def train_epoch(model, dataloader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    pbar = tqdm(dataloader, desc='Training')
    for pixel_values, labels in pbar:
        pixel_values, labels = pixel_values.to(device), labels.to(device)
        optimizer.zero_grad()
        logits = model(pixel_values)
        loss = criterion(logits, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = logits.max(1)
        total += labels.size(0)
        correct += predicted.eq(labels).sum().item()
        all_preds.extend(predicted.cpu().numpy())
        all_labels.extend(labels.cpu().numpy())
        pbar.set_postfix({
            'loss': f'{running_loss/(pbar.n+1):.4f}',
            'acc': f'{100.*correct/total:.2f}%'
        })
    _, _, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)
    return running_loss / len(dataloader), 100. * correct / total, f1

def validate(model, dataloader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    all_preds = []
    all_labels = []
    with torch.no_grad():
        pbar = tqdm(dataloader, desc='Validation')
        for pixel_values, labels in pbar:
            pixel_values, labels = pixel_values.to(device), labels.to(device)
            logits = model(pixel_values)
            loss = criterion(logits, labels)
            running_loss += loss.item()
            _, predicted = logits.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()
            all_preds.extend(predicted.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())
            pbar.set_postfix({
                'loss': f'{running_loss/(pbar.n+1):.4f}',
                'acc': f'{100.*correct/total:.2f}%'
            })
    _, _, f1, _ = precision_recall_fscore_support(all_labels, all_preds, average='weighted', zero_division=0)
    return running_loss / len(dataloader), 100. * correct / total, f1, all_preds, all_labels

def calculate_metrics(y_true, y_pred):
    """Calculate comprehensive metrics including sensitivity and specificity"""
    cm = confusion_matrix(y_true, y_pred)
    
    # For binary classification
    tn, fp, fn, tp = cm.ravel()
    
    # Sensitivity (Recall) for each class
    sensitivity_class0 = tn / (tn + fn) if (tn + fn) > 0 else 0  # Normal
    sensitivity_class1 = tp / (tp + fp) if (tp + fp) > 0 else 0  # Abnormal
    
    # Specificity for each class
    specificity_class0 = tn / (tn + fp) if (tn + fp) > 0 else 0  # Normal
    specificity_class1 = tp / (tp + fn) if (tp + fn) > 0 else 0  # Abnormal
    
    # Average
    avg_sensitivity = (sensitivity_class0 + sensitivity_class1) / 2
    avg_specificity = (specificity_class0 + specificity_class1) / 2
    
    # Overall metrics
    precision, recall, f1, _ = precision_recall_fscore_support(y_true, y_pred, average='weighted')
    accuracy = accuracy_score(y_true, y_pred)
    
    return {
        'accuracy': accuracy * 100,
        'precision': precision,
        'recall': recall,
        'f1': f1,
        'sensitivity_class0': sensitivity_class0,
        'sensitivity_class1': sensitivity_class1,
        'specificity_class0': specificity_class0,
        'specificity_class1': specificity_class1,
        'avg_sensitivity': avg_sensitivity,
        'avg_specificity': avg_specificity,
        'confusion_matrix': cm
    }

# ============================================================================
# MAIN
# ============================================================================

def main():
    print("="*70)
    print("ViT + CNN-2D-CBAM + Fusion + Quantum Hybrid Network")
    print("ERBMAHE Dataset Classification")
    print("Binary Classification: Abnormal vs Normal")
    print(f"METHOD 2: K-FOLD ({N_FOLDS} folds) WITH SEPARATE TEST SET (10%)")
    print("="*70)

    data_path = 'D:/training/archive/ICMR_datasets_ERBMAHE'

    print(f"\n🤖 Base Model: {MODEL_NAME}")
    print(f"⚛️  Quantum Qubits: {N_QUBITS}")
    print(f"📊 Classes: {CLASSES}")
    print(f"🔄 K-Fold: {N_FOLDS} folds on 90% data")
    print(f"🧪 Test Set: 10% holdout")

    print("\n📁 Loading dataset...")
    data_list = []
    for class_name in CLASSES:
        class_path = os.path.join(data_path, class_name)
        if os.path.exists(class_path):
            for img_file in os.listdir(class_path):
                if img_file.lower().endswith(('.png', '.jpg', '.jpeg', '.bmp', '.tiff')):
                    data_list.append({
                        'image_path': os.path.join(class_path, img_file),
                        'label': class_name,
                        'class_id': CLASSES.index(class_name)
                    })
        else:
            print(f"⚠️ Warning: {class_path} not found!")
    
    df = pd.DataFrame(data_list)
    print(f"📊 Total images: {len(df)}")
    print("\nClass distribution:")
    print(df['label'].value_counts())

    # Split: 90% for k-fold, 10% for final test
    kfold_df, test_df = train_test_split(df, test_size=0.10, stratify=df['class_id'], random_state=42)
    
    print(f"\n📊 Dataset split:")
    print(f"  K-Fold data: {len(kfold_df)} ({len(kfold_df)/len(df)*100:.1f}%)")
    print(f"  Test data:   {len(test_df)} ({len(test_df)/len(df)*100:.1f}%)")

    # Class weights for focal loss (calculated on k-fold data)
    class_counts = kfold_df['label'].value_counts().sort_index().values
    total_samples = len(kfold_df)
    class_weights = total_samples / (len(CLASSES) * class_counts)
    class_weights = torch.FloatTensor(class_weights).to(device)
    print(f"\n⚖️ Class weights for Focal Loss:")
    for i, class_name in enumerate(CLASSES):
        print(f"  {class_name}: {class_weights[i]:.4f}")

    # ViT processor
    print(f"\n🔧 Loading ViT processor...")
    processor = ViTImageProcessor.from_pretrained(MODEL_NAME)

    # K-Fold setup
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    
    fold_results = []
    all_fold_histories = []
    
    batch_size = 16
    num_epochs = 50

    print(f"\n⚙️ Training configuration:")
    print(f"  Max epochs per fold: {num_epochs}")
    print(f"  Batch size: {batch_size}")
    print(f"  Early stopping patience: 10 epochs")
    print(f"  Learning rate (ViT): 1e-5")
    print(f"  Learning rate (CNN): 2e-4")
    print(f"  Learning rate (Quantum): 5e-5")
    print(f"  Learning rate (Classifier): 1e-4")
    print(f"  Scheduler: CosineAnnealingLR")
    print(f"  Loss: Focal Loss (gamma=2.0)")
    print(f"  Strategy: Method 2 with separate test set")

    # ============================================================================
    # K-FOLD CROSS-VALIDATION
    # ============================================================================
    
    for fold_idx, (train_idx, val_idx) in enumerate(skf.split(kfold_df, kfold_df['class_id'])):
        print("\n" + "="*70)
        print(f"FOLD {fold_idx + 1}/{N_FOLDS}")
        print("="*70)
        
        train_df = kfold_df.iloc[train_idx].reset_index(drop=True)
        val_df = kfold_df.iloc[val_idx].reset_index(drop=True)
        
        print(f"\n📊 Fold {fold_idx + 1} split:")
        print(f"  Train:      {len(train_df)} ({len(train_df)/len(kfold_df)*100:.1f}% of k-fold data)")
        print(f"  Validation: {len(val_df)} ({len(val_df)/len(kfold_df)*100:.1f}% of k-fold data)")
        
        train_dataset = ERBMAHEDataset(train_df, processor=processor, augment=True)
        val_dataset = ERBMAHEDataset(val_df, processor=processor, augment=False)
        
        train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, pin_memory=True)
        val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
        
        print(f"\n🤖 Creating model for Fold {fold_idx + 1}...")
        model = ViTQuantumHybrid(model_name=MODEL_NAME, num_classes=len(CLASSES), n_qubits=N_QUBITS).to(device)
        
        for p in model.parameters():
            p.requires_grad = True
        
        if fold_idx == 0:
            total_params = sum(p.numel() for p in model.parameters())
            trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
            quantum_params = sum(p.numel() for p in model.quantum_layer.parameters())
            
            print(f"✓ Total parameters: {total_params:,}")
            print(f"✓ Trainable parameters: {trainable_params:,}")
            print(f"✓ Quantum layer parameters: {quantum_params:,}")
            print(f"✓ Model size: {total_params * 4 / 1024 / 1024:.2f} MB")
        
        def build_optimizer(model):
            groups = []
            vit_params = [p for p in model.vit.parameters() if p.requires_grad]
            if vit_params:
                groups.append({'params': vit_params, 'lr': 1e-5})
            cnn_params = [p for p in model.cnn_branch.parameters() if p.requires_grad]
            if cnn_params:
                groups.append({'params': cnn_params, 'lr': 2e-4})
            q_params = [p for p in model.quantum_layer.parameters() if p.requires_grad]
            if q_params:
                groups.append({'params': q_params, 'lr': 5e-5})
            clf_params = [p for p in model.classifier.parameters() if p.requires_grad]
            if clf_params:
                groups.append({'params': clf_params, 'lr': 1e-4})
            if not groups:
                groups = [{'params': [p for p in model.parameters() if p.requires_grad], 'lr': 1e-4}]
            return torch.optim.AdamW(groups, weight_decay=0.01)
        
        criterion = FocalLoss(alpha=class_weights, gamma=2.0, reduction='mean')
        optimizer = build_optimizer(model)
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs, eta_min=1e-6)
        early_stopping = EarlyStopping(patience=10, min_delta=0.001, mode='max', verbose=True)
        
        best_val_acc = 0.0
        fold_history = {
            'train_loss': [], 'train_acc': [], 'train_f1': [],
            'val_loss': [], 'val_acc': [], 'val_f1': []
        }
        
        # Training loop
        for epoch in range(num_epochs):
            print(f"\nFold {fold_idx + 1} - Epoch {epoch+1}/{num_epochs}")
            print("-" * 70)
            
            train_loss, train_acc, train_f1 = train_epoch(model, train_loader, criterion, optimizer, device)
            val_loss, val_acc, val_f1, _, _ = validate(model, val_loader, criterion, device)
            
            scheduler.step()
            
            fold_history['train_loss'].append(train_loss)
            fold_history['train_acc'].append(train_acc)
            fold_history['train_f1'].append(train_f1)
            fold_history['val_loss'].append(val_loss)
            fold_history['val_acc'].append(val_acc)
            fold_history['val_f1'].append(val_f1)
            
            current_lr = optimizer.param_groups[0]['lr']
            print(f"\nResults:")
            print(f"  Train - Loss: {train_loss:.4f}, Acc: {train_acc:.2f}%, F1: {train_f1:.4f}")
            print(f"  Val   - Loss: {val_loss:.4f}, Acc: {val_acc:.2f}%, F1: {val_f1:.4f}")
            print(f"  Learning Rate (group0): {current_lr:.2e}")
            
            if val_acc > best_val_acc:
                best_val_acc = val_acc
                torch.save(model.state_dict(), f'fold{fold_idx+1}_best.pth')
                print(f"  💾 Best model saved! (Val Acc: {best_val_acc:.2f}%)")
            
            if early_stopping(val_acc, epoch):
                print(f"\n⚠️ Early stopping at epoch {epoch+1}")
                break
        
        # Final validation evaluation
        print(f"\n📊 EVALUATING FOLD {fold_idx + 1}")
        print("="*70)
        model.load_state_dict(torch.load(f'fold{fold_idx+1}_best.pth'))
        val_loss, val_acc, val_f1, y_pred, y_true = validate(model, val_loader, criterion, device)
        
        metrics = calculate_metrics(y_true, y_pred)
        
        print(f"\n📈 Fold {fold_idx + 1} Validation Results:")
        print(f"  Accuracy:         {metrics['accuracy']:.2f}%")
        print(f"  Precision:        {metrics['precision']:.4f}")
        print(f"  Recall:           {metrics['recall']:.4f}")
        print(f"  F1 Score:         {metrics['f1']:.4f}")
        print(f"  Sensitivity (Normal):   {metrics['sensitivity_class0']:.4f}")
        print(f"  Sensitivity (Abnormal): {metrics['sensitivity_class1']:.4f}")
        print(f"  Specificity (Normal):   {metrics['specificity_class0']:.4f}")
        print(f"  Specificity (Abnormal): {metrics['specificity_class1']:.4f}")
        print(f"  Avg Sensitivity:  {metrics['avg_sensitivity']:.4f}")
        print(f"  Avg Specificity:  {metrics['avg_specificity']:.4f}")
        
        fold_results.append({
            'fold': fold_idx + 1,
            'val_acc': metrics['accuracy'],
            'val_precision': metrics['precision'],
            'val_recall': metrics['recall'],
            'val_f1': metrics['f1'],
            'val_sensitivity': metrics['avg_sensitivity'],
            'val_specificity': metrics['avg_specificity'],
            'best_val_acc': best_val_acc,
            'epochs_trained': len(fold_history['train_loss']),
            'model_path': f'fold{fold_idx+1}_best.pth',
            'y_true': y_true,
            'y_pred': y_pred,
            'metrics': metrics
        })
        
        all_fold_histories.append(fold_history)
        
        # Save confusion matrix
        cm = metrics['confusion_matrix']
        plt.figure(figsize=(8, 6))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=CLASSES, yticklabels=CLASSES)
        plt.title(f'Fold {fold_idx + 1} - Confusion Matrix\n(Val Acc: {metrics["accuracy"]:.2f}%)')
        plt.ylabel('True Label')
        plt.xlabel('Predicted Label')
        plt.tight_layout()
        plt.savefig(f'fold{fold_idx+1}_confusion_matrix.png', dpi=150)
        plt.close()
        print(f"✓ Confusion matrix saved")
        
        # Enhanced visualization for FIRST FOLD
        if fold_idx == 0:
            print(f"\n📊 Creating enhanced visualizations for Fold 1...")
            
            # 1. Training/Validation curves
            fig, axes = plt.subplots(1, 3, figsize=(18, 5))
            epochs_range = range(1, len(fold_history['train_loss']) + 1)
            
            axes[0].plot(epochs_range, fold_history['train_loss'], 'b-o', label='Train', linewidth=2, markersize=4)
            axes[0].plot(epochs_range, fold_history['val_loss'], 'r-s', label='Val', linewidth=2, markersize=4)
            axes[0].set_xlabel('Epoch', fontsize=12)
            axes[0].set_ylabel('Loss', fontsize=12)
            axes[0].set_title('Fold 1 - Loss', fontsize=14, fontweight='bold')
            axes[0].legend(fontsize=10)
            axes[0].grid(True, alpha=0.3)
            
            axes[1].plot(epochs_range, fold_history['train_acc'], 'b-o', label='Train', linewidth=2, markersize=4)
            axes[1].plot(epochs_range, fold_history['val_acc'], 'r-s', label='Val', linewidth=2, markersize=4)
            axes[1].set_xlabel('Epoch', fontsize=12)
            axes[1].set_ylabel('Accuracy (%)', fontsize=12)
            axes[1].set_title('Fold 1 - Accuracy', fontsize=14, fontweight='bold')
            axes[1].legend(fontsize=10)
            axes[1].grid(True, alpha=0.3)
            
            axes[2].plot(epochs_range, fold_history['train_f1'], 'b-o', label='Train', linewidth=2, markersize=4)
            axes[2].plot(epochs_range, fold_history['val_f1'], 'r-s', label='Val', linewidth=2, markersize=4)
            axes[2].set_xlabel('Epoch', fontsize=12)
            axes[2].set_ylabel('F1 Score', fontsize=12)
            axes[2].set_title('Fold 1 - F1 Score', fontsize=14, fontweight='bold')
            axes[2].legend(fontsize=10)
            axes[2].grid(True, alpha=0.3)
            
            plt.tight_layout()
            plt.savefig('fold1_training_curves.png', dpi=150)
            plt.close()
            print("  ✓ Training curves saved")
            
            # 2. Detailed confusion matrix
            fig, ax = plt.subplots(figsize=(10, 8))
            cm_percent = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
            annotations = np.empty_like(cm).astype(str)
            for i in range(cm.shape[0]):
                for j in range(cm.shape[1]):
                    annotations[i, j] = f'{cm[i, j]}\n({cm_percent[i, j]:.1f}%)'
            sns.heatmap(cm, annot=annotations, fmt='', cmap='Blues', 
                       xticklabels=CLASSES, yticklabels=CLASSES,
                       cbar_kws={'label': 'Count'}, ax=ax)
            ax.set_title(f'Fold 1 - Detailed Confusion Matrix\nVal Acc: {metrics["accuracy"]:.2f}%', 
                        fontsize=14, fontweight='bold')
            ax.set_ylabel('True Label', fontsize=12)
            ax.set_xlabel('Predicted Label', fontsize=12)
            plt.tight_layout()
            plt.savefig('fold1_confusion_detailed.png', dpi=150)
            plt.close()
            print("  ✓ Detailed confusion matrix saved")
            
            # 3. Metrics summary
            fig, ax = plt.subplots(figsize=(12, 6))
            metric_names = ['Accuracy', 'Precision', 'Recall', 'F1 Score', 'Sensitivity', 'Specificity']
            metric_values = [
                metrics['accuracy'],
                metrics['precision']*100,
                metrics['recall']*100,
                metrics['f1']*100,
                metrics['avg_sensitivity']*100,
                metrics['avg_specificity']*100
            ]
            colors = ['#3498db', '#2ecc71', '#e74c3c', '#f39c12', '#9b59b6', '#1abc9c']
            bars = ax.bar(metric_names, metric_values, color=colors, alpha=0.7, edgecolor='black', linewidth=1.5)
            for bar, value in zip(bars, metric_values):
                height = bar.get_height()
                ax.text(bar.get_x() + bar.get_width()/2., height,
                       f'{value:.2f}%', ha='center', va='bottom', fontsize=11, fontweight='bold')
            ax.set_ylabel('Score (%)', fontsize=12)
            ax.set_title('Fold 1 - Validation Metrics Summary', fontsize=14, fontweight='bold')
            ax.set_ylim([0, 105])
            ax.grid(True, alpha=0.3, axis='y')
            plt.xticks(rotation=15)
            plt.tight_layout()
            plt.savefig('fold1_metrics_summary.png', dpi=150)
            plt.close()
            print("  ✓ Metrics summary saved")
    
    # ============================================================================
    # SUMMARY ACROSS FOLDS
    # ============================================================================
    
    print("\n" + "="*70)
    print("📊 K-FOLD CROSS-VALIDATION SUMMARY")
    print("="*70)
    
    accuracies = [r['val_acc'] for r in fold_results]
    precisions = [r['val_precision'] for r in fold_results]
    recalls = [r['val_recall'] for r in fold_results]
    f1_scores = [r['val_f1'] for r in fold_results]
    sensitivities = [r['val_sensitivity'] for r in fold_results]
    specificities = [r['val_specificity'] for r in fold_results]
    
    print(f"\n📈 Cross-Validation Results ({N_FOLDS} folds):")
    print(f"  Accuracy:     {np.mean(accuracies):.2f}% ± {np.std(accuracies):.2f}%")
    print(f"  Precision:    {np.mean(precisions):.4f} ± {np.std(precisions):.4f}")
    print(f"  Recall:       {np.mean(recalls):.4f} ± {np.std(recalls):.4f}")
    print(f"  F1 Score:     {np.mean(f1_scores):.4f} ± {np.std(f1_scores):.4f}")
    print(f"  Sensitivity:  {np.mean(sensitivities):.4f} ± {np.std(sensitivities):.4f}")
    print(f"  Specificity:  {np.mean(specificities):.4f} ± {np.std(specificities):.4f}")
    
    # Find best fold
    best_fold_idx = np.argmax(accuracies)
    best_fold = fold_results[best_fold_idx]
    print(f"\n🏆 Best Fold: {best_fold['fold']} (Val Acc: {best_fold['val_acc']:.2f}%)")
    
    # ============================================================================
    # EVALUATE BEST MODEL ON TEST SET
    # ============================================================================
    
    print("\n" + "="*70)
    print("🧪 EVALUATING BEST MODEL ON HOLDOUT TEST SET")
    print("="*70)
    
    print(f"\n📊 Loading best model from Fold {best_fold['fold']}...")
    best_model = ViTQuantumHybrid(model_name=MODEL_NAME, num_classes=len(CLASSES), n_qubits=N_QUBITS).to(device)
    best_model.load_state_dict(torch.load(best_fold['model_path']))
    
    test_dataset = ERBMAHEDataset(test_df, processor=processor, augment=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=0, pin_memory=True)
    
    test_loss, test_acc, test_f1, test_pred, test_true = validate(best_model, test_loader, criterion, device)
    test_metrics = calculate_metrics(test_true, test_pred)
    
    print(f"\n📈 Test Set Results (Best Model from Fold {best_fold['fold']}):")
    print(f"  Accuracy:         {test_metrics['accuracy']:.2f}%")
    print(f"  Precision:        {test_metrics['precision']:.4f}")
    print(f"  Recall:           {test_metrics['recall']:.4f}")
    print(f"  F1 Score:         {test_metrics['f1']:.4f}")
    print(f"  Sensitivity (Normal):   {test_metrics['sensitivity_class0']:.4f}")
    print(f"  Sensitivity (Abnormal): {test_metrics['sensitivity_class1']:.4f}")
    print(f"  Specificity (Normal):   {test_metrics['specificity_class0']:.4f}")
    print(f"  Specificity (Abnormal): {test_metrics['specificity_class1']:.4f}")
    print(f"  Avg Sensitivity:  {test_metrics['avg_sensitivity']:.4f}")
    print(f"  Avg Specificity:  {test_metrics['avg_specificity']:.4f}")
    
    # Test confusion matrix
    test_cm = test_metrics['confusion_matrix']
    plt.figure(figsize=(10, 8))
    test_cm_percent = test_cm.astype('float') / test_cm.sum(axis=1)[:, np.newaxis] * 100
    test_annotations = np.empty_like(test_cm).astype(str)
    for i in range(test_cm.shape[0]):
        for j in range(test_cm.shape[1]):
            test_annotations[i, j] = f'{test_cm[i, j]}\n({test_cm_percent[i, j]:.1f}%)'
    sns.heatmap(test_cm, annot=test_annotations, fmt='', cmap='Greens', 
               xticklabels=CLASSES, yticklabels=CLASSES,
               cbar_kws={'label': 'Count'})
    plt.title(f'Test Set Confusion Matrix\n(Best Model from Fold {best_fold["fold"]}, Acc: {test_metrics["accuracy"]:.2f}%)', 
             fontsize=14, fontweight='bold')
    plt.ylabel('True Label', fontsize=12)
    plt.xlabel('Predicted Label', fontsize=12)
    plt.tight_layout()
    plt.savefig('test_confusion_matrix.png', dpi=150)
    plt.close()
    print("\n✓ Test confusion matrix saved")
    
    # Save all results
    results_df = pd.DataFrame(fold_results)
    results_df.to_csv('kfold_results.csv', index=False)
    print("✓ Results saved to CSV")
    
    print("\n" + "="*70)
    print("✅ TRAINING COMPLETE")
    print("="*70)
    print(f"\n📊 Final Summary:")
    print(f"  Dataset: ERBMAHE (Abnormal vs Normal)")
    print(f"  Strategy: Method 2 K-Fold with 10% Test Set")
    print(f"  K-Fold CV Accuracy: {np.mean(accuracies):.2f}% ± {np.std(accuracies):.2f}%")
    print(f"  Test Set Accuracy:  {test_metrics['accuracy']:.2f}%")
    print(f"  Test Set F1 Score:  {test_metrics['f1']:.4f}")
    print("\n" + "="*70)

if __name__ == '__main__':
    main()

✓ PyTorch & torchvision imported successfully
✓ Transformers imported successfully
✓ PennyLane imported successfully
🔧 Using device: cuda
🔧 PyTorch version: 2.5.1+cu121
ViT + CNN-2D-CBAM + Fusion + Quantum Hybrid Network
ERBMAHE Dataset Classification
Binary Classification: Abnormal vs Normal
METHOD 2: K-FOLD (5 folds) WITH SEPARATE TEST SET (10%)

🤖 Base Model: google/vit-base-patch16-224
⚛️  Quantum Qubits: 4
📊 Classes: ['Abnormal', 'Normal']
🔄 K-Fold: 5 folds on 90% data
🧪 Test Set: 10% holdout

📁 Loading dataset...
📊 Total images: 1981

Class distribution:
label
Normal      1270
Abnormal     711
Name: count, dtype: int64

📊 Dataset split:
  K-Fold data: 1782 (90.0%)
  Test data:   199 (10.0%)

⚖️ Class weights for Focal Loss:
  Abnormal: 1.3922
  Normal: 0.7802

🔧 Loading ViT processor...

⚙️ Training configuration:
  Max epochs per fold: 50
  Batch size: 16
  Early stopping patience: 10 epochs
  Learning rate (ViT): 1e-5
  Learning rate (CNN): 2e-4
  Learning rate (Quantum): 5e-5


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Total parameters: 98,885,421
✓ Trainable parameters: 98,885,421
✓ Quantum layer parameters: 6,921
✓ Model size: 377.22 MB

Fold 1 - Epoch 1/50
----------------------------------------------------------------------


Validation: 100%|██████████| 23/23 [00:41<00:00,  1.82s/it, loss=0.0537, acc=94.40%]



Results:
  Train - Loss: 0.0947, Acc: 84.56%, F1: 0.8465
  Val   - Loss: 0.0537, Acc: 94.40%, F1: 0.9439
  Learning Rate (group0): 9.99e-06
  💾 Best model saved! (Val Acc: 94.40%)
  ✓ Initial best score: 94.3978

Fold 1 - Epoch 2/50
----------------------------------------------------------------------


...


Results:
  Train - Loss: 0.0016, Acc: 99.79%, F1: 0.9979
  Val   - Loss: 0.0165, Acc: 98.60%, F1: 0.9860
  Learning Rate (group0): 4.11e-06
  No improvement. Patience: 10/10

⚠️ Early stopping triggered!
   Best score: 99.1573 at epoch 19

⚠️ Early stopping at epoch 30

📊 EVALUATING FOLD 5


Validation: 100%|██████████| 23/23 [00:25<00:00,  1.11s/it, loss=0.0108, acc=99.16%]



📈 Fold 5 Validation Results:
  Accuracy:         99.16%
  Precision:        0.9916
  Recall:           0.9916
  F1 Score:         0.9916
  Sensitivity (Normal):   0.9921
  Sensitivity (Abnormal): 0.9913
  Specificity (Normal):   0.9844
  Specificity (Abnormal): 0.9956
  Avg Sensitivity:  0.9917
  Avg Specificity:  0.9900
✓ Confusion matrix saved

📊 K-FOLD CROSS-VALIDATION SUMMARY

📈 Cross-Validation Results (5 folds):
  Accuracy:     99.05% ± 0.38%
  Precision:    0.9905 ± 0.0037
  Recall:       0.9905 ± 0.0038
  F1 Score:     0.9905 ± 0.0038
  Sensitivity:  0.9889 ± 0.0053
  Specificity:  0.9905 ± 0.0029

🏆 Best Fold: 2 (Val Acc: 99.72%)

🧪 EVALUATING BEST MODEL ON HOLDOUT TEST SET

📊 Loading best model from Fold 2...


Some weights of ViTModel were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Validation: 100%|██████████| 13/13 [00:19<00:00,  1.47s/it, loss=0.0243, acc=96.48%]


📈 Test Set Results (Best Model from Fold 2):
  Accuracy:         96.48%
  Precision:        0.9656
  Recall:           0.9648
  F1 Score:         0.9650
  Sensitivity (Normal):   0.9324
  Sensitivity (Abnormal): 0.9840
  Specificity (Normal):   0.9718
  Specificity (Abnormal): 0.9609
  Avg Sensitivity:  0.9582
  Avg Specificity:  0.9664

✓ Test confusion matrix saved
✓ Results saved to CSV

✅ TRAINING COMPLETE

📊 Final Summary:
  Dataset: ERBMAHE (Abnormal vs Normal)
  Strategy: Method 2 K-Fold with 10% Test Set
  K-Fold CV Accuracy: 99.05% ± 0.38%
  Test Set Accuracy:  96.48%
  Test Set F1 Score:  0.9650

